# ENGR 418 Project - Stage 2
## Sorting lego blocks using image data

- Group #20
- Jesse Alele 82807728
- Zach Kelly 41637836

### Import Statements

In [7]:
import numpy as np 
import matplotlib.pyplot as plt
import os
from numpy import asarray
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from PIL import Image
from skimage import feature

### Feature Extraction

In [17]:
# def feature_extractor(path):
folder = r"C:\Users\zachk\Documents\ubco\year5\ubc_term1\engr418\project\stage2\lego_dataset_2\training"

im_size = 100
file_names = os.listdir(folder)
x = []

for i in range(len(file_names)):
    path = folder + '/' + file_names[i] # update path as we iterate through file_names array
    im = Image.open(path).convert('L') # open ith image and convert to grayscale
    #im_array = asarray(im) # convert to numpy array
    im = im.resize((im_size,im_size)) # resize to specified pixel length and width

    im = feature.canny(image=im,sigma=5,low_threshold=0.001,high_threshold=0.07)
    #im_height = im.shape[0]
    #im_width = im.shape[1]
    #display(im_height,im_width)


AttributeError: dtype

### Label Extraction

### Classifier

### Model Training

### Accuracy on Training Set

### Accuracy on Test Set

Define function to get training and testing data from raw image files

In [2]:
im_width = int(np.sqrt(4067)) 
im_length = int(np.sqrt(4067))

def get_data(folder, im_width, im_length):

    # create list of file names inside folder
    file_names = os.listdir(folder)
    # intialize np array to store feature data
    x = np.empty((len(file_names),im_width*im_length))
    # initialize np array to store class labels
    y = np.empty((len(file_names),1))
    
    num_cir = 0 # number of images that are labelled as class circle
    num_rec = 0 # number of images that are labelled as class rectangle
    num_squ = 0 # number of images that are labelled as class square
    
    for i in range(len(file_names)):
       
        # fill class labels array
        if 'cir' in file_names[i]:
            y[i] = 0 # class circle
            num_cir = num_cir + 1 # keep running count of circle labels
        elif 'rec' in file_names[i]:
            y[i] = 1 # class rectangle
            num_rec = num_rec + 1 # keep running count of rectangle labels
        elif 'squ' in file_names[i]:
            y[i] = 2 # class square
            num_squ = num_squ + 1 # keep running count of square labels
            
        # fill feature array
        path = folder + '/' + file_names[i] # update path as we iterate through file_names array
        im = Image.open(path).convert('L') # open ith image and convert to grayscale
        im = im.resize((im_length,im_width)) # resize to specified pixel length and width
        im_array = asarray(im) # convert to numpy array
        x[i,:] = im_array.reshape(1,-1) # reshape to im_length*im_width columns by one row then insert as ith row in x
    
    y = np.ravel(y)

    # assure the folder contains an equal count of each different shape, so only one variable (num_shape) is needed to seperate the shapes in class labels array
    if num_cir == num_rec == num_squ:
        num_shape = num_cir
    else:
        print('The number of circles, rectangles and squares in the specified folder are not equal')

    return x, y, num_shape